In [1]:
import json 
with open('iranianNamesDataset.json', 'r', encoding="utf-8") as f:
    word_dict = json.load(f)
words = []
for name in word_dict['RECORDS']:
    words.append(name['Names'])
words

['ابوالحسن',
 'احمد',
 'احمدرضا',
 'آذر',
 'آرش',
 'آ رمان',
 'آ ریا',
 'ا زاده',
 'آ زاد',
 'ا سماعیل',
 'اشرف',
 'ا صغر',
 'ا عظم',
 'افسانه',
 'ا فشین',
 'ا قدس',
 'ا کبر',
 'آ کو',
 'آ لان',
 'ا لشان',
 'ا لنا ز',
 'ا لناز',
 'ا لها م',
 'ا لهه',
 'ا م البنین',
 'ا میر',
 'ا میر حسین',
 'ا میر ر ضا',
 'آ مینه',
 'آ ناهید',
 'ا نیس',
 'آوات',
 'آ ی تک',
 'ادریس',
 'اسدالله',
 'اصغر',
 'اکبر',
 'الله بردی',
 'الهام',
 'ام لیلا',
 'امید',
 'ایرج',
 'ابا',
 'ابا بکر',
 'ابا ذر',
 'ابا سلت',
 'ابا صلت',
 'ابابکر',
 'ابات',
 'آباد',
 'آبادان',
 'ابادر',
 'اباذر',
 'اباذر علی',
 'ابراهیم',
 'اباز',
 'ابازر',
 'اباس',
 'اباسط',
 'اباسعد',
 'اباسل',
 'اباسلت',
 'اباسلط',
 'اباسلک',
 'اباصت',
 'اباصد',
 'اباصل',
 'اباصلت',
 'اباصلط',
 'ابالفتح',
 'ابالفضل',
 'آبامه',
 'آبان',
 'آبان دخت',
 'اباهیم',
 'آبت اله',
 'ابتاج',
 'ابتسام',
 'ابتسلم',
 'ابتصام',
 'ابتهاج',
 'ابتین',
 'آبتین آی',
 'آبث',
 'ابجد',
 'ابحش',
 'ابخش',
 'ابدال',
 'ابدالحسن',
 'ابدالرضا',
 'ابدالفضل',
 'ابدالمجید',
 'ابدالو

In [2]:
import torch
device = torch.device("mps")
x = torch.arange(1, device=device)
print (x)


tensor([0], device='mps:0')


In [3]:
chars = sorted(list(set(''.join(words))))
Abakolah = chars[5]
chars = chars[9:-1]
chars.insert(0, Abakolah)
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [4]:
xs, ys = [], []

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        try:
            ix1 = stoi[ch1]
            ix2 = stoi[ch2]
            xs.append(ix1)
            ys.append(ix2)
        except:
            pass
xs = torch.tensor(xs, device = device)
ys = torch.tensor(ys, device = device)

In [5]:
import torch.nn.functional as F
xenc = F.one_hot(xs, 38).float()

In [6]:
g = torch.Generator(device = device)
W = torch.randn((38, 38), generator=g,requires_grad=True, device = device)
logits = xenc @ W
counts = logits.exp()
probability = counts / counts.sum(1, keepdim = True)

In [7]:
nlls = torch.zeros(9, device = device)
for i in range(9):
    x = xs[i].item()
    y = ys[i].item()
    print(f'x = {x}, y = {y}')
    p = probability[i, y]
    logp = torch.log(p)
    nll = -logp
    print(nll)
    nlls[i] = nll
nlls.mean().item()

x = 0, y = 2
tensor(3.1600, device='mps:0', grad_fn=<NegBackward0>)
x = 2, y = 3
tensor(5.9349, device='mps:0', grad_fn=<NegBackward0>)
x = 3, y = 28
tensor(3.9446, device='mps:0', grad_fn=<NegBackward0>)
x = 28, y = 2
tensor(4.0687, device='mps:0', grad_fn=<NegBackward0>)
x = 2, y = 24
tensor(4.0026, device='mps:0', grad_fn=<NegBackward0>)
x = 24, y = 7
tensor(4.4119, device='mps:0', grad_fn=<NegBackward0>)
x = 7, y = 13
tensor(3.9079, device='mps:0', grad_fn=<NegBackward0>)
x = 13, y = 26
tensor(2.8015, device='mps:0', grad_fn=<NegBackward0>)
x = 26, y = 0
tensor(3.2507, device='mps:0', grad_fn=<NegBackward0>)


3.942521333694458

In [8]:
logits = xenc @ W #log counts
counts = logits.exp()
probability = counts / counts.sum(1, keepdim = True)

In [ ]:
num = xs.nelement()
for k in range(5000):
    xenc = F.one_hot(xs, num_classes = 38).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims = True)
    loss = - probs[torch.arange(num), ys].log().mean()
    if k % 500 == 0: print(loss.item())

    W.grad = None
    loss.backward()
    
    W.data -= 5 * W.grad

4.06204080581665
2.6559038162231445


In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(20,20),  )
plt.imshow(W.detach().cpu().numpy(), cmap = "Blues")
for i in range(38):
    for j in range(38):
        chstr = itos[i] + '->' + itos[j]
        plt.text(i, j, chstr, ha="center", va="bottom", color='black')
        #print("A")
        plt.text(i, j, f'{ W[i, j].item():0.2f}', ha="center", va="top", color='gray')



In [24]:
for i in range(10):
    name = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix], device=device), num_classes = 38).float()
        logitx = xenc @ W
        counts = logitx.exp()
        p = counts / counts.sum(1, keepdim = True)
        ix = torch.multinomial(p, replacement= True, generator= g, num_samples = 1).item()
        if ix == 0:
            name.append('end')
            break
        name.append(itos[ix])
    print(name)

['ن', 'ی', 'م', 'ا', 'ر', 'end']
['ب', 'ی', 'ر', 'ا', 'م', 'end']
['م', 'د', 'س', 'ی', 'end']
['ع', 'ل', 'ی', 'ه', 'end']
['ا', 'ر', 'و', 'ی', 'end']
['م', 'end']
['م', 'ح', 'م', 'ح', 'م', 'د', 'ر', 'و', 'ر', 'ت', 'و', 'ش', 'ک', 'ب', 'ا', 'ب', 'د', 'ا', 'ی', 'end']
['م', 'ی', 'end']
['ه', 'end']
['د', 'end']
